In [ ]:
class Particle:
    def __init__(self, num_features):
        self.position = np.random.randint(0, 2, num_features)  # Position is a binary vector (0 or 1)
        self.velocity = np.random.uniform(-1, 1, num_features)
        self.best_position = self.position.copy()
        self.best_value = float('-inf')  # Start with the worst possible value

    def update_velocity(self, global_best_position, w, c1, c2):
        r1, r2 = np.random.rand(len(self.position)), np.random.rand(len(self.position))
        cognitive_component = c1 * r1 * (self.best_position - self.position)
        social_component = c2 * r2 * (global_best_position - self.position)
        self.velocity = w * self.velocity + cognitive_component + social_component

    def update_position(self):
        sigmoid = lambda x: 1 / (1 + np.exp(-x))
        probability = sigmoid(self.velocity)
        randoms = np.random.rand(len(self.position))
        self.position = np.where(randoms < probability, 1, 0)

        # Ensure at least one feature is selected
        if not self.position.any():
            self.position[np.random.randint(0, len(self.position))] = 1import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.base import clone
import pyswarms as ps
import random

def preprocess(data):
    for column in data.columns:
        if data[column].dtype == 'object':
            if len(data[column].unique()) == 2:
                data[column] = pd.Categorical(data[column]).codes
            else:
                le = LabelEncoder()
                data[column] = le.fit_transform(data[column])
    return data

def feature_selection_pso(data, target_variable, model, seed=None):
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)

    data = preprocess(data)
    X = data.drop(columns=[target_variable])
    y = data[target_variable]
    feature_names = X.columns

    # Definir la función objetivo
    def objective_function(particle):
        # Seleccionar características según la posición de la partícula
        mask = particle >= 0.5
        if np.sum(mask) == 0:
            return 1.0  # Peor score si no selecciona ninguna característica
        X_selected = X.iloc[:, mask]
        model_clone = clone(model)
        score = -np.mean(cross_val_score(model_clone, X_selected, y, cv=5, scoring='accuracy'))
        return score

    # Adaptar la función objetivo para múltiples partículas
    def objective_function_swarm(swarm):
        return np.array([objective_function(particle) for particle in swarm])

    # Configuración de PSO
    n_particles = 30
    dimensions = X.shape[1]
    options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

    # Inicialización del optimizador
    optimizer = ps.discrete.BinaryPSO(n_particles=n_particles, dimensions=dimensions, options=options)

    # Optimización
    cost, pos = optimizer.optimize(objective_function_swarm, iters=50)

    # Seleccionar las características óptimas
    selected_features = feature_names[pos >= 0.5]
    print("Características seleccionadas:", list(selected_features))

    return selected_features

# Ejemplo de uso
if __name__ == "__main__":
    # Parámetros de entrada
    dataset_path = 'ruta/al/dataset.csv'  # Reemplaza con la ruta a tu dataset
    target_variable = 'nombre_variable_objetivo'  # Reemplaza con el nombre de tu variable objetivo
    seed = 42  # Opcional: establece la semilla para reproducibilidad

    # Cargar el dataset
    data = pd.read_csv(dataset_path)

    # Definir el modelo
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(random_state=seed)

    # Ejecutar la selección de características
    selected_features = feature_selection_pso(data, target_variable, model, seed=seed)

    # Preparar los datos con las características seleccionadas
    data_selected = data[selected_features]
    target = data[target_variable]

    # Ahora puedes usar data_selected y target para entrenar tu modelo
    model.fit(data_selected, target)


ModuleNotFoundError: No module named 'pyswarms'